In [3]:
from DataReader import DataReaderChallangePolimi2017
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from CB_asy import CB_asymmetric_cosine
import numpy as np
import copy
import  scipy.sparse as sps
from sklearn.utils.sparsefuncs import inplace_row_scale
from sklearn.utils.sparsefuncs import inplace_column_scale
from sklearn.preprocessing import normalize

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
t_p = dr.target_playlists
t_t = dr.target_tracks
#slim bpr
s_bpr = dr.load_sparse_csr("./DumpData/sim_bpr_20e_100k_fulltrain.npz")

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


IOError: [Errno 2] No such file or directory: './DumpData/sim_bpr_100e_100k_fulltrain.npz'

In [2]:
#content based
m_cb = CB_asymmetric_cosine(urm, icm, k=150, alpha=0.11, q=1, m=0)
r_cb = m_cb.getRM_csr(targetPlaylists=t_p,targetTracks=t_t)
if evaluation: dr.evaluateMAP(r_cb)

Start  CB asy model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0642


In [109]:
#collaborative filterin item based
m_cf_ib = CF_IB_asymmetric_cosine(urm, k=200, alpha=0.30, q=1, m=0)
r_cf_ib = m_cf_ib.getRM_csr(targetPlaylists=t_p,targetTracks=t_t)
if evaluation: dr.evaluateMAP(r_cf_ib)

Start asymmetric cosine item-based model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model asymmetric cosine item-based done


In [110]:
#collaborative filtering user based
m_cf_ub = CF_IB_asymmetric_cosine(urm, k=200, alpha=0.45, q=1, m=0)
r_cf_ub = m_cf_ub.getRM_csr(targetPlaylists=t_p,targetTracks=t_t)
if evaluation: dr.evaluateMAP(r_cf_ub)

Start asymmetric cosine item-based model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model asymmetric cosine item-based done


In [111]:
r_bpr = urm * s_bpr
if evaluation: score = dr.evaluateMAP(r_bpr)

In [24]:
r = 1.1*r_cb + 2.55 * r_cf_ib + 1.1*r_cf_ub
if evaluation: score = dr.evaluateMAP(r)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0903


In [57]:
r = 1.1*r_cb + 2.55 * r_cf_ib + 2.55 * r_bpr  + 1.1*r_cf_ub 
if evaluation: score = dr.evaluateMAP(r)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0915


In [330]:
maps1, n1 = dr.evaluateMAP(r_cb,allmaps=True)
maps2, n2 = dr.evaluateMAP(r_cf_ib,allmaps=True)
maps3, n3 = dr.evaluateMAP(r_cf_ub,allmaps=True)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0621
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0726
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0661


In [332]:
recs = dr.getAllRecommendations(r)
dr.evaluateMAPfromRecommendations(recs)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0883


0.088326000000000626

In [333]:
count = 0 
p_cb = {}

th = 0.0
for row in maps1:
    if(row[1]>=th):
        count = count +1
        p_cb[row[0]] = [row[1],row[2]]

count2 = 0 
p_cf_ib = {}
for row in maps2:
    if(row[1]>=th):
        count2 = count2 +1
        p_cf_ib[row[0]] = [row[1],row[2]]

print "cb: " + str(count)
print "cf: " + str(count2)

cb: 10000
cf: 10000


In [279]:
join = 0 
th2 = 0.15
for p in p_cb.keys():
    if p in p_cf_ib.keys():
        join = join +1
        m_cb = p_cb[p][0]
        m_cf = p_cf_ib[p][0]
        if m_cb - m_cf >th2:
            p_cf_ib.pop(p, None)
        elif m_cf - m_cb > th2:
            p_cb.pop(p,None)
        else:
            p_cf_ib.pop(p, None)
            p_cb.pop(p,None)
print "common: " + str(join)
print "new cb: " + str(len(p_cb))
print "new cf: " + str(len (p_cf_ib))

recs_c = copy.deepcopy(recs)
for row in recs_c:
    p = row[0]
    if p in p_cb:
        row[1] = p_cb[p][1]
    if p in p_cf_ib:
        row[1] = p_cf_ib[p][1]

common: 10000
new cb: 837
new cf: 1140


In [280]:
dr.evaluateMAPfromRecommendations(recs_c)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.1101


0.11007050000000086

In [281]:
join = 0
for p in p_cb.keys():
    if p not in p_cb_old.keys():
        join = join +1
        p_cb.pop(p,None)
print join

join = 0
for p in p_cf_ib.keys():
    if p not in p_cf_ib_old.keys():
        join = join +1
        p_cf_ib.pop(p, None)
print join
print "new cb: " + str(len(p_cb))
print "new cf: " + str(len (p_cf_ib))

507
674
new cb: 330
new cf: 466


In [342]:
recs_c = copy.deepcopy(recs)
for row in recs_c:
    p = row[0]
    if p in p_cb_old_s1:############
        row[1] = p_cb[p][1]
    if p in p_cf_ib_old_s1:############
        row[1] = p_cf_ib[p][1]
dr.evaluateMAPfromRecommendations(recs_c)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0915


0.091484388888889448

In [253]:
p_cb_old_s1 #= copy.deepcopy(p_cb)
p_cf_ib_old_s1 #= copy.deepcopy(p_cf_ib)

In [343]:
print len(p_cb_old)
print len(p_cf_ib_old)
print len(p_cb_old_s1)
print len(p_cf_ib_old_s1)
print len(p_cb_old_s)
print len(p_cf_ib_old_s)
print len(p_cb_old_s2)
print len(p_cf_ib_old_s2)


827
1058
327
434
196
263
130
178


In [114]:
import pickle

#pickle.dump(p_cb_old_s1, open( "./DumpData/top_playlists_cb", "wb" ))
#pickle.dump(p_cf_ib_old_s1, open( "./DumpData/top_playlists_cf_ib", "wb" ))

l1=pickle.load(open( "./DumpData/top_playlists_cb", "rb" ))
l2=pickle.load(open( "./DumpData/top_playlists_cf_ib", "rb" ))

In [ ]:
rec = dr.getAllRecommendations(r_norm)
rec1 = dr.getAllRecommendations(r_cb_norm)
rec2 = dr.getAllRecommendations(r_cf_ib_norm)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done
User 0 of 10000


In [118]:
count = 0 
p_cb = {}

th = 0.0
for row in rec1:
    count = count +1
    p_cb[row[0]] = row[1]

count2 = 0 
p_cf_ib = {}
for row in rec2:
    count2 = count2 +1
    p_cf_ib[row[0]] = row[1]

print "cb: " + str(count)
print "cf: " + str(count2)

for row in rec:
    p = row[0]
    if p in l1:############
        row[1] = p_cb[p]
    if p in l2:############
        row[1] = p_cf_ib[p]
if evaluation: dr.evaluateMAPfromRecommendations(rec)

cb: 10000
cf: 10000


In [112]:
r_cb_norm = normalize(r_cb,axis=1)
r_cf_ib_norm = normalize(r_cf_ib,axis=1)
r_cf_ub_norm = normalize(r_cf_ub,axis=1)
r_bpr_norm = normalize (r_bpr,axis = 1)

In [113]:
r_norm = 0.85*r_cb_norm + r_cf_ib_norm  + 1.1*r_cf_ub_norm + 0.36 * r_bpr_norm
r_norm = normalize(r_norm,axis=1)
if evaluation: score = dr.evaluateMAP(r_norm)


In [58]:
rec5 = dr.getAllRecommendations(r_norm)
rec1 = dr.getAllRecommendations(r_cb)
rec2 = dr.getAllRecommendations(r_cf_ib)
rec3 = dr.getAllRecommendations(r_cf_ub)
rec4 = dr.getAllRecommendations(r_bpr)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommendations done


In [59]:
def mix2 (all_r1,all_r2, k = 1):
    all_r = []
    for i in range(len(all_r1)):
        id_p = all_r1[i][0]
        row1 = all_r1[i][1]
        row2 = all_r2[i][1]
        row = np.array(np.append(row1,row2))
        sc1 = all_r1[i][2]
        sc2 = all_r2[i][2]
        sc2 = np.multiply(sc2,k)
        #print "1: "+ str(sc1)
        #print "2: " + str(sc2)
        sc = np.array(np.append(sc1,sc2))
        idxs = sc.argsort()[::-1]
        rec = np.array(row[idxs])
        sc = np.array(sc[idxs])
        l = []
        s = []
        for j in range(10):
            if np.asscalar(rec[j]) not in l:
                l.append(rec[j])
                s.append(sc[j])
        l = np.array(l[:5])
        s = np.array(s[:5])
        all_r.append([id_p,l,s])
        
    return all_r

In [86]:
rec_t = rec2
if evaluation: dr.evaluateMAPfromRecommendations(rec_t)
rec_t = mix2(rec_t,rec4,0.6)
if evaluation: dr.evaluateMAPfromRecommendations(rec_t)
rec_t = mix2(rec_t,rec1,0.2)
if evaluation: dr.evaluateMAPfromRecommendations(rec_t)
rec_t = mix2(rec_t,rec3,0.7)
if evaluation: dr.evaluateMAPfromRecommendations(rec_t)
rec_t = mix2(rec_t,rec4,0.7)
if evaluation: dr.evaluateMAPfromRecommendations(rec_t)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0754
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0757
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0803
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0800
User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0804


In [119]:
if not evaluation: dr.buildSubmissionFile(file_name="cfib-cbaa-cfub-sslim",recommendation=rec)

Creating submission file in: ./Submissions/cfib-cbaa-cfub-sslim
Done


In [4]:
def find_parameters():   
    result = []
    for k in range (225,301,5):
        rec = CF_IB_asymmetric_cosine(urm, k=k, alpha=0.3, q=1, m=0)
        r = rec.getRM_csr()
        MAP = dr.evaluateMAP(r)
        rec.MAP = MAP
        result.append (rec.print_setting())
    print ("\n\nResults:\n\n")
    for r in result:
        print (r)